In [1]:
import os # Necessário para verificar se os arquivos existem
import pandas as pd
from icalendar import Calendar
from datetime import datetime, date, time

In [2]:
def ical_to_dataframe(file_path, prefix):
    """
    Convert iCal file to DataFrame with specified column prefix
    
    Args:
        file_path (str): Path to iCal file
        prefix (str): Prefix for column names (e.g., 'OTA' or 'Google')
    """
    events = []
    
    try:
        with open(file_path, 'rb') as f:
            cal = Calendar.from_ical(f.read())
            
        for event in cal.walk('VEVENT'):
            start = event.get('dtstart').dt
            end = event.get('dtend').dt
            summary = str(event.get('summary'))
            
            # Convert to datetime if date
            if isinstance(start, date) and not isinstance(start, datetime):
                start = datetime.combine(start, time(hour=16))
            if isinstance(end, date) and not isinstance(end, datetime):
                end = datetime.combine(end, time(hour=11))
                
            events.append({
                f'Origem': prefix,
                f'Inicio': start,
                f'Fim': end,
                f'Summary': summary
            })
            
        return pd.DataFrame(events)
    
    except Exception as e:
        print(f"Erro ao processar {file_path}: {str(e)}")
        return pd.DataFrame()



In [3]:
import pandas as pd
from datetime import datetime

def filtrar_reservas_passadas(df_original, data_corte):
    """
    Filtra um DataFrame, removendo todas as linhas onde a data
    na coluna 'Fim' é anterior (menor que) à data_corte fornecida.

    Args:
        df_original (pd.DataFrame): O DataFrame para filtrar (deve ter a coluna 'Fim').
        data_corte (str or datetime): A data de referência (cutoff).
                                      Linhas com 'Fim' < data_corte serão removidas.

    Returns:
        pd.DataFrame: Um novo DataFrame contendo apenas as linhas onde
                      'Fim' >= data_corte, ou None se ocorrer um erro.
    """
    
    # 1. Validações Iniciais
    if 'Fim' not in df_original.columns:
        print("ERRO: O DataFrame não contém a coluna 'Fim'.")
        return None
        
    print(f"Iniciando filtro. (Tamanho original: {len(df_original)} linhas)")
    
    # 2. Faz uma cópia para segurança (evita SettingWithCopyWarning)
    df_filtrado = df_original.copy()

    # 3. Converte a data_corte para um objeto datetime (Timestamp)
    try:
        # pd.to_datetime é robusto e lida com strings ou objetos datetime
        data_corte_dt = pd.to_datetime(data_corte, errors='raise')
    except Exception as e:
        print(f"ERRO: A 'data_corte' fornecida ('{data_corte}') é inválida: {e}")
        return None
    
    # 4. Garante que a coluna 'Fim' no DataFrame seja datetime
    # errors='coerce' transforma datas inválidas (ex: 'Em Andamento') em NaT
    df_filtrado['Fim'] = pd.to_datetime(df_filtrado['Fim'], errors='coerce')

    # 5. Remove linhas com datas 'Fim' inválidas (que viraram NaT)
    linhas_antes = len(df_filtrado)
    df_filtrado.dropna(subset=['Fim'], inplace=True)
    linhas_removidas_nat = linhas_antes - len(df_filtrado)
    if linhas_removidas_nat > 0:
        print(f"  Aviso: Removidas {linhas_removidas_nat} linhas com data 'Fim' inválida ou vazia.")

    # 6. Aplica o filtro
    # A condição para MANTER as linhas é 'Fim' >= 'data_corte'
    linhas_antes_filtro = len(df_filtrado)
    
    condicao_manter = df_filtrado['Fim'] >= data_corte_dt
    
    df_resultado = df_filtrado[condicao_manter]
    
    linhas_removidas_passadas = linhas_antes_filtro - len(df_resultado)
    print(f"  Removidas {linhas_removidas_passadas} linhas com 'Fim' anterior a {data_corte_dt.strftime('%d/%m/%Y %H:%M')}.")
    print(f"Filtro concluído. (Tamanho final: {len(df_resultado)} linhas)")

    return df_resultado


In [4]:
import pandas as pd
from datetime import datetime

def filtrar_reservas_apos_1_ano(df_original, coluna_data='Inicio'):
    """
    Filtra um DataFrame para remover reservas que começam
    daqui a mais de um ano.

    Args:
        df_original (pd.DataFrame): DataFrame com as reservas.
        coluna_data (str, optional): Nome da coluna de data (ex: 'Inicio') 
                                     para usar como base do filtro. 
                                     Default é 'Inicio'.

    Returns:
        pd.DataFrame: DataFrame filtrado (apenas com linhas até 1 ano no futuro),
                      ou None se a coluna de data não existir.
    """
    
    # 1. Verificar se a coluna de data existe
    if coluna_data not in df_original.columns:
        print(f"ERRO: A coluna '{coluna_data}' não foi encontrada no DataFrame.")
        return None # Retorna None em caso de erro

    df = df_original.copy()
    linhas_originais = len(df)
    print(f"Iniciando filtro de 1 ano (linhas originais: {linhas_originais})...")

    # 2. Definir as datas de corte
    # Usamos .normalize() para pegar a meia-noite de hoje (início do dia)
    hoje = pd.to_datetime('today').normalize() 
    # Define a data limite (exatamente 1 ano a partir de hoje)
    data_limite = hoje + pd.DateOffset(years=1)

    print(f"  Data de hoje (base): {hoje.strftime('%d/%m/%Y')}")
    print(f"  Data limite (corte): {data_limite.strftime('%d/%m/%Y')}")

    # 3. Garantir que a coluna de data é datetime
    # errors='coerce' transforma datas inválidas em NaT (Not a Time)
    # (Se já for datetime, não faz mal)
    df[coluna_data] = pd.to_datetime(df[coluna_data], errors='coerce')

    # 4. Remover linhas com datas inválidas (NaT)
    linhas_antes_nat = len(df)
    df.dropna(subset=[coluna_data], inplace=True)
    linhas_removidas_nat = linhas_antes_nat - len(df)
    if linhas_removidas_nat > 0:
        print(f"  Aviso: Removidas {linhas_removidas_nat} linhas com data '{coluna_data}' inválida ou vazia.")

    # 5. Aplicar o filtro
    # Queremos MANTER linhas onde a data é MENOR OU IGUAL à data limite
    condicao_manter = df[coluna_data] <= data_limite
    
    df_resultado = df[condicao_manter]

    linhas_removidas_futuro = len(df) - len(df_resultado)
    print(f"  Removidas {linhas_removidas_futuro} linhas com '{coluna_data}' posterior a {data_limite.strftime('%d/%m/%Y')}.")
    print(f"Filtro concluído. (Tamanho final: {len(df_resultado)} linhas)")

    return df_resultado



In [5]:
import pandas as pd

def remover_duplicatas_por_data(df_original):
    """
    Remove TODAS as linhas de um DataFrame que compartilham
    os mesmos valores nas colunas 'Inicio' e 'Fim'.

    Se duas (ou mais) linhas têm o mesmo 'Inicio' e 'Fim',
    todas essas linhas são excluídas. Apenas linhas com
    combinações únicas de 'Inicio'/'Fim' são mantidas.

    Args:
        df_original (pd.DataFrame): DataFrame com colunas 
                                    'Origem', 'Inicio', 'Fim', 'Summary'.

    Returns:
        pd.DataFrame: Um novo DataFrame contendo apenas as linhas
                      com pares 'Inicio'/'Fim' únicos.
    """
    
    # Colunas chave para verificar duplicatas
    #colunas_chave = ['Inicio', 'Fim']
    colunas_chave = ['Fim']
    
    # 1. Verifica se as colunas necessárias existem
    if not all(col in df_original.columns for col in colunas_chave):
        print(f"ERRO: O DataFrame não contém as colunas necessárias: {colunas_chave}")
        return df_original # Retorna original se não puder processar

    df = df_original.copy()
    linhas_originais = len(df)
    
    print(f"Iniciando verificação de duplicatas (linhas originais: {linhas_originais})...")

    # 2. Garante que as colunas de data sejam datetime para comparação correta
    #    (Converte strings para datetime, se já forem datetime, não faz mal)
    #    errors='coerce' transforma datas inválidas em NaT (Not a Time)
    try:
        df['Inicio'] = pd.to_datetime(df['Inicio'], errors='coerce')
        df['Fim'] = pd.to_datetime(df['Fim'], errors='coerce')
    except Exception as e:
        print(f"  Aviso: Falha ao converter colunas de data (podem já ser datetime). Erro: {e}")
        # Continua mesmo assim, pois duplicated() pode funcionar

    # 3. Identifica TODAS as linhas que fazem parte de um grupo duplicado
    #    keep=False marca TODAS as ocorrências de duplicatas como True.
    #    Linhas únicas (incluindo NaTs únicos) são marcadas como False.
    duplicados_mask = df.duplicated(subset=colunas_chave, keep=False)

    # 4. Filtra o DataFrame, mantendo apenas as linhas que NÃO (~)
    #    são parte de um grupo duplicado (ou seja, onde a máscara é False).
    df_resultado = df[~duplicados_mask]
    
    linhas_removidas = linhas_originais - len(df_resultado)
    print(f"Processamento concluído. {linhas_removidas} linhas duplicadas foram removidas.")
    
    return df_resultado

In [6]:
import pandas as pd
from datetime import datetime

def encontrar_data_fim_mais_antiga_ota(df_reservas):
    """
    Encontra a data mais antiga (mínima) na coluna 'Fim'
    para todas as reservas cuja 'Origem' é 'OTA'.

    Args:
        df_reservas (pd.DataFrame): Um DataFrame que deve conter
                                   as colunas 'Origem' e 'Fim'.

    Returns:
        pd.Timestamp or pd.NaT: A data mais antiga (mínima) encontrada.
                                Retorna pd.NaT (Not a Time) se nenhuma
                                reserva 'OTA' for encontrada ou se as
                                datas forem inválidas.
    """
    print("Iniciando busca pela data de 'Fim' mais antiga da 'OTA'...")
    
    # --- 1. Verificação de Colunas ---
    colunas_necessarias = ['Origem', 'Fim']
    if not all(col in df_reservas.columns for col in colunas_necessarias):
        print(f"ERRO: O DataFrame não contém as colunas necessárias ({colunas_necessarias}).")
        return pd.NaT

    try:
        # --- 2. Filtrar por 'OTA' ---
        # Seleciona apenas as linhas onde 'Origem' é 'OTA'
        df_ota = df_reservas[df_reservas['Origem'] == 'OTA']

        # --- 3. Verificar se algo foi encontrado ---
        if df_ota.empty:
            print("Aviso: Nenhuma reserva com 'Origem' = 'OTA' foi encontrada.")
            return pd.NaT

        # --- 4. Garantir que 'Fim' é datetime ---
        # Converte a coluna 'Fim' para datetime
        # errors='coerce' transforma datas inválidas em NaT (Not a Time)
        datas_fim_ota = pd.to_datetime(df_ota['Fim'], errors='coerce')

        # --- 5. Encontrar a data mais antiga (mínima) ---
        data_mais_antiga = datas_fim_ota.min()

        if pd.isna(data_mais_antiga):
            print("Aviso: Reservas 'OTA' encontradas, mas nenhuma data de 'Fim' válida.")
            return pd.NaT
            
        print(f"Data mais antiga encontrada: {data_mais_antiga}")
        return data_mais_antiga

    except Exception as e:
        print(f"ERRO inesperado durante o processamento: {e}")
        return pd.NaT

# --- Exemplo de Uso ---

# # 1. Criar um DataFrame de exemplo
# data = {
#     'Origem': ['OTA', 'Google', 'Direto', 'OTA', 'OTA'],
#     'Inicio': [
#         '2025-11-01', '2025-10-25', '2025-10-28', '2025-10-20', '2025-11-05'
#     ],
#     'Fim': [
#         '2025-11-10', '2025-10-27', '2025-10-30', '2025-10-24', '2025-11-08'
#     ],
#     'Summary': ['Reserva A', 'Reserva B', 'Reserva C', 'Reserva D', 'Reserva E']
# }
# df_exemplo = pd.DataFrame(data)
# 
# # Adicionar uma data inválida
# df_exemplo.loc[len(df_exemplo)] = ['OTA', '2025-12-01', 'Data Inválida', 'Reserva F']

# print("--- DataFrame de Exemplo ---")
# display(df_exemplo)

# # 2. Chamar a função
# data_antiga = encontrar_data_fim_mais_antiga_ota(df_exemplo)

# if pd.notna(data_antiga):
#     print(f"\nResultado Final: A data de 'Fim' mais antiga da 'OTA' é: {data_antiga.strftime('%d/%m/%Y')}")
# else:
#     print("\nResultado Final: Não foi possível determinar a data.")

In [7]:
print("--- Iniciando verificação de inconsistências ---")

# 1. Defina a lista de apartamentos para iterar
apartment_list = [
    'c108', 
    'd014', 
    'cbl004',
    'ap101',
    'ap201',
    'f216',
]

# 2. Itere sobre a lista
for apt in apartment_list:
    print(f"\n--- Verificando Apartamento: {apt.upper()} ---")

    # 3. Defina os caminhos dos arquivos
    file_ota = f'calendars/{apt}_merged_booking_airbnb.ics'
    file_google = f'calendars/{apt}_google.ics'

    # 4. Verifique se os arquivos necessários existem
    if not os.path.exists(file_ota) or not os.path.exists(file_google):
        print(f"Aviso: Pulando {apt.upper()}. Um ou mais arquivos .ics não foram encontrados.")
        if not os.path.exists(file_ota):
            print(f"  -> {file_ota} (Não encontrado)")
        if not os.path.exists(file_google):
            print(f"  -> {file_google} (Não encontrado)")
        continue # Pula para o próximo apartamento no loop

    # 5. Execute a comparação (os arquivos existem)
    try:
        # Convert both files to DataFrames
        df_ota = ical_to_dataframe(file_ota, 'OTA')
        df_google = ical_to_dataframe(file_google, 'Google')
        df_final = pd.concat([df_ota, df_google], ignore_index=True)
        
        data_corte = encontrar_data_fim_mais_antiga_ota(df_final)

        df_final = filtrar_reservas_passadas(df_final, data_corte)
        
        # 2. Chamar a função (usando a coluna padrão 'Inicio')
        df_final = filtrar_reservas_apos_1_ano(df_final)
        
        df_final = remover_duplicatas_por_data(df_final)
        
        
        if df_final is None:
             print(f"Função remover_duplicatas_por_data retornou 'None' para {apt.upper()}. Pulando.")
             continue

        inconsistencies = df_final

        # 6. Exiba os resultados
        if inconsistencies is not None and not inconsistencies.empty:
            print(f"=== Encontradas {len(inconsistencies)} inconsistências em {apt.upper()} ===")
            display(inconsistencies)
            print(f"=== Salvando em {apt}_inconsistencias.csv ===")
            inconsistencies.to_csv(f'calendars/{apt}_inconsistencias.csv', index=False, quoting=1)
        else:
            print(f"=== Nenhuma inconsistência encontrada em {apt.upper()} ===")
    
    except Exception as e:
        print(f"ERRO: Ocorreu uma exceção inesperada ao processar {apt.upper()}: {e}")

print("\n--- Verificação de todos os apartamentos concluída! ---")

--- Iniciando verificação de inconsistências ---

--- Verificando Apartamento: C108 ---
Iniciando busca pela data de 'Fim' mais antiga da 'OTA'...
Data mais antiga encontrada: 2025-12-19 11:00:00
Iniciando filtro. (Tamanho original: 58 linhas)
  Removidas 40 linhas com 'Fim' anterior a 19/12/2025 11:00.
Filtro concluído. (Tamanho final: 18 linhas)
Iniciando filtro de 1 ano (linhas originais: 18)...
  Data de hoje (base): 13/12/2025
  Data limite (corte): 13/12/2026
  Removidas 0 linhas com 'Inicio' posterior a 13/12/2026.
Filtro concluído. (Tamanho final: 18 linhas)
Iniciando verificação de duplicatas (linhas originais: 18)...
Processamento concluído. 18 linhas duplicadas foram removidas.
=== Nenhuma inconsistência encontrada em C108 ===

--- Verificando Apartamento: D014 ---
Iniciando busca pela data de 'Fim' mais antiga da 'OTA'...
Data mais antiga encontrada: 2025-12-18 11:00:00
Iniciando filtro. (Tamanho original: 30 linhas)
  Removidas 12 linhas com 'Fim' anterior a 18/12/2025 11:

,Origem,Inicio,Fim,Summary
0,OTA,2025-12-11 16:00:00,2025-12-15 11:00:00,Airbnb
1,OTA,2025-12-20 16:00:00,2025-12-23 11:00:00,Airbnb
2,OTA,2025-12-23 16:00:00,2025-12-30 11:00:00,Airbnb
3,OTA,2026-01-05 16:00:00,2026-01-09 11:00:00,Airbnb
4,OTA,2025-12-15 16:00:00,2025-12-20 11:00:00,Direto
5,OTA,2026-01-09 16:00:00,2026-01-11 11:00:00,Direto
6,OTA,2026-05-01 16:00:00,2026-05-04 11:00:00,Direto


=== Salvando em f216_inconsistencias.csv ===

--- Verificação de todos os apartamentos concluída! ---
